# NSRR HomePAP Features Extraction

Note that we can only use in-lab recording because the at-home recording do not have the necessary channels.

We also only use the "full" night and not the split-lab (during which the participants proceeded with CPAP titration).

**WARNING:** 

1) C4 is C4-FPZ.

https://sleepdata.org/datasets/homepap/pages/montage-and-sampling-rate-information.md

In [ ]:
import os
import os
import yasa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf

root_dir = '/Volumes/NSRR/homepap/'
eeg_dir = root_dir + 'polysomnography/edfs/lab/full/'
hypno_dir = root_dir + 'polysomnography/annotations-events-profusion/lab/full/'

desc_dir = root_dir + 'datasets/homepap-baseline-dataset-0.1.0.csv'

df_subj = pd.read_csv(desc_dir, usecols=['nsrrid', 'treatmentarm', 'age', 'gender', 'bmi', 'ahi_full',
                                         'race3', 'ethnicity'])

df_subj.rename(columns={
    'nsrrid': 'subj',
    'race3': 'race',
    'ahi_full': 'ahi',
    }, inplace=True)

df_subj['race'].replace({1: 'caucasian', 2: 'african', 3: 'other'}, inplace=True)
df_subj.loc[df_subj['ethnicity'] == 1, 'race'] = 'hispanic'
df_subj.drop(columns=['ethnicity'], inplace=True)
df_subj.rename(columns={'race': 'ethnicity'}, inplace=True)

df_subj['male'] = df_subj['gender'].replace({'M': 1, 'F': 0})

# Drop NaN
df_subj.dropna(inplace=True)

# Keep only in-lab PSG
df_subj = df_subj[df_subj['treatmentarm'] == 'Lab']
df_subj.drop(columns=['treatmentarm', 'gender'], inplace=True)

# Keep only full night
files = [int(f.split('-')[3]) for f in os.listdir(hypno_dir)]
good = np.intersect1d(files, df_subj['subj'].values, assume_unique=True)
df_subj = df_subj[df_subj['subj'].isin(good)]

# Convert to str
df_subj['subj'] = df_subj['subj'].astype(str)
df_subj.set_index('subj', inplace=True)

# Export demographics to CSV file
df_subj['dataset'] = 'HOMEPAP'
df_subj.to_csv("/Volumes/JAWA/SHERLOCK/demo/demo_nsrr_homepap.csv")

print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

In [ ]:
df = []
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'homepap-lab-full-' + sub + '.edf'
    hypno_file = hypno_dir + 'homepap-lab-full-' + sub + '-profusion.xml'

    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        chan = raw.info['ch_names']
        # Try different combinations of channels
        # Do not delete! Channels have different names in HomePAP.
        eeg_chan = np.intersect1d(chan, ['C4-M1', 'C4'])[0]
        loc_chan = np.intersect1d(chan, ['E1', 'E-1', 'L-EOG'])[0]
        emg_chan = np.intersect1d(chan, ['Lchin', 'LChin', 'Chin1-Chin2', 'EMG1', 'LCHIN'])[0]
        include = [eeg_chan, loc_chan, emg_chan]
        raw = read_raw_edf(
            eeg_file, preload=True, exclude=np.setdiff1d(chan, include), verbose=0)
    except:
        continue
        
    # Resample and low-pass filter 
    raw.resample(sf, npad="auto")
    
    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # We keep up to 15 minutes before / after sleep
    start_to_firstsleep_min = np.nonzero(hypno)[0][0] / 2
    lastsleep = np.nonzero(hypno)[0][-1]
    lastsleep_to_end_min = (len(hypno) - lastsleep) / 2
    tmin, tmax = 0, None  # must be in seconds
    if start_to_firstsleep_min > 15:
        tmin = (start_to_firstsleep_min - 15) * 60
    if lastsleep_to_end_min > 15:
        tmax = lastsleep * 30 + 15 * 60
    # Crop!
    raw.crop(tmin, tmax)
    if tmax is None:
        hypno = hypno[int(tmin / 60 * 2):]
    else:
        hypno = hypno[int(tmin / 60 * 2):int(tmax / 60 * 2)]
    # Hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA
    # Hypnogram must include all stages
    if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
        print("- Not all stages are present.")
        continue
    # If the duration is not between 4 to 12 hours, skip subject
    if not(4 < n_epochs / 120 < 12):
        print("- Recording too short/long.")
        continue
    # Requires at least 5 min of each stage
    # if (stage_min < 5).any():
    #    print("- Not 5 min of each stage.")
    #    continue
       
    # EXTRACT FEATURES
    metadata = dict(age=df_subj.loc[sub, 'age'], male=df_subj.loc[sub, 'male'])
    sls = yasa.SleepStaging(raw, eeg_name=include[0], eog_name=include[1], 
                            emg_name=include[2], metadata=metadata)

    features = sls.get_features().reset_index()
    features['subj'] = sub
    features.set_index(['subj', 'epoch'], inplace=True)
    
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    df.append(features)

df = pd.concat(df)

In [ ]:
# Add dataset
df['dataset'] = 'homepap'

# Convert to category
df['dataset'] = df['dataset'].astype('category')
df['stage'] = df['stage'].astype('category')

In [ ]:
# Show %stage
df['stage'].value_counts(normalize=True, sort=True)

In [ ]:
# Number of unique nights in dataset
df.index.get_level_values(0).nunique()

In [ ]:
# Median value of the EEG IQR per stage
df.groupby('stage')['eeg_iqr'].median()

In [ ]:
# Export to Parquet
df.to_parquet("/Volumes/JAWA/SHERLOCK/features/features_nsrr_homepap.parquet")